На базе баскетбольных матчей добейтесь средней абсолютной ошибки 17 и менее очков.

## **Подключение библиотек**

In [ ]:
# Работа с массивами данных
import numpy as np

# Работа с табличными данными
import pandas as pd

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential, Model

# Основные слои
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation, Input, concatenate
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D

# Оптимизаторы
from tensorflow.keras.optimizers import Adam, Adadelta, SGD, Adagrad, RMSprop

# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Масштабирование данных
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.callbacks import ReduceLROnPlateau
#from tensorflow.keras.utils import plot_history

# Загрузка датасетов из облака google
import gdown

# Регулярные выражения
import re

# Отрисовка графиков
import matplotlib.pyplot as plt

# Метрики для расчета ошибок
from sklearn.metrics import mean_squared_error, mean_absolute_error

%matplotlib inline

##**Подготовка**

In [ ]:
# Загрузка из google облака
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l10/basketball.csv', None, quiet=True)

df = pd.read_csv('basketball.csv', encoding= 'cp1251', sep=';', header=0, index_col=0) # Загружаем базу
df.head(20)

,TOTAL,info,Ком. 1,Ком. 2,Минута,Общая минута,Секунда,fcount,ftime
0,"98,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,0.0,1,1.0,30,81,90.0
1,"100,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,1,1.0,45,81,105.0
2,"99,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,2,2.0,0,81,120.0
3,"98,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,2,2.0,30,81,150.0
4,"95,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,3,3.0,0,81,180.0
5,"91,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,4,4.0,0,81,240.0
6,"93,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,4,4.0,4,4.0,30,81,270.0
7,"90,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,4,4.0,6,6.0,0,81,360.0
8,"91,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,5,4.0,6,6.0,0,81,360.0
9,"92,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,5,5.0,6,6.0,0,81,360.0


Извлекаем текстовые данные из колонки `info` таблицы, помещаем в переменную `data_text`. Выводим длину списка:

In [ ]:
data_text = df['info'].values #

len(data_text) #

52450

In [ ]:
data_text

array(['4081445 Новая Зеландия. Женщины. WBC. Регулярный сезон. Нельсон(Нельсон Спаркс - Токерау Феоникс)\n',
       '4081445 Новая Зеландия. Женщины. WBC. Регулярный сезон. Нельсон(Нельсон Спаркс - Токерау Феоникс)\n',
       '4081445 Новая Зеландия. Женщины. WBC. Регулярный сезон. Нельсон(Нельсон Спаркс - Токерау Феоникс)\n',
       ...,
       '4163481 Португалия. Мужчины. LPB. Плей-офф. 1/4 финала(Галитуш Баррейру - Оливейренсе)\n',
       '4163481 Португалия. Мужчины. LPB. Плей-офф. 1/4 финала(Галитуш Баррейру - Оливейренсе)\n',
       '4163481 Португалия. Мужчины. LPB. Плей-офф. 1/4 финала(Галитуш Баррейру - Оливейренсе)\n'],
      dtype=object)

Задаем максимальное кол-во слов в словаре, помещаем в переменную все символы, которые хотим вычистить из текста.

 Токенизируем текстовые данные:

In [ ]:
maxWordsCount = 5000

sim_for_del='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'

tokenizer = Tokenizer (num_words=maxWordsCount,
                       filters=sim_for_del,
                       lower=True,
                       split=' ',
                       oov_token='unknown',
                       char_level=False)

tokenizer.fit_on_texts(data_text)

In [ ]:
# Переводим в Эмбеддинг пространство
Sequences = tokenizer.texts_to_sequences(data_text)

# Вариант  Bag of Words
xBOW_text = tokenizer.sequences_to_matrix(Sequences)

Преобразуем данные в numpy, подготовим наборы для обучения:

In [ ]:
xTrain = np.array(df[['Ком. 1','Ком. 2', 'Минута', 'Секунда','ftime']].astype('int'))
yTrain = np.array(df['fcount'].astype('int'))

In [ ]:
print(xTrain.shape)
print(yTrain.shape)
print(xBOW_text.shape)

(52450, 5)
(52450,)
(52450, 5000)


In [ ]:
# Функция по проверке ошибки

def check_MAE_predictl_DubbleInput (model,
                                    x_data,
                                    x_data_text,
                                    y_data_not_scaled,
                                    plot=False):

  mae = 0 # Инициализируем начальное значение ошибки
  y_pred = (model.predict([x_data,x_data_text])).squeeze()

  for n in range (0,len(x_data)):
    mae += abs(y_data_not_scaled[n] - y_pred[n]) # Увеличиваем значение ошибки для текущего элемента
  mae /= len(x_data) # Считаем среднее значение
  print('Среднаяя абслолютная ошибка {:.3f} очков это {:.3f}% от общей выборки в {} игры'.format(mae, (mae/y_data_not_scaled.mean(axis=0))*100,len(x_data)))

  if plot:
     plt.scatter(y_data_not_scaled, y_pred)
     plt.xlabel('Правильные значение')
     plt.ylabel('Предсказания')
     plt.axis('equal')
     plt.xlim(plt.xlim())
     plt.ylim(plt.ylim())
     plt.plot([0, 250], [0, 250])
     plt.show()

In [ ]:
def on_epoch_end_custom(epoch, logs=None):
    check_MAE_predictl_DubbleInput(model_final_scaled,xTrain_scaled,xBOW_text,yTrain,plot=True)

## **Функция оценки**

In [ ]:
# Функция оценки результатов и вывода оценки

def eval_net(model, x_train, y_train, y_scaler = None, n = 10, limit = 1000.):

    pred = model.predict(x_train)             # Предсказание на тренировочной выборке
    if y_scaler:                              # Если есть нормирование - то денормировать
        pred = y_scaler.inverse_transform(pred)

    print('Средняя абсолютная ошибка:', mean_absolute_error(pred, y_train), '\n')

    for i in range(n):
        print('Реальное значение: {:6.2f}  Предсказанное значение: {:6.2f}  Разница: {:6.2f}'.format(y_train[i, 0],
                                                                                                    pred[i, 0],
                                                                                                    abs(y_train[i, 0] - pred[i, 0])))
    # Построение картинки разброса предсказанных и верных значений зарплаты
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.scatter(y_train, pred)                 # Отрисовка точечного графика
    ax.set_xlim(0, limit)                     # Пределы по x, y
    ax.set_ylim(0, limit)
    ax.plot(plt.xlim(), plt.ylim(), 'r')      # Отрисовка диагональной линии
    plt.xlabel('Правильные значения')
    plt.ylabel('Предсказания')
    plt.grid()
    plt.show()

## **Обучение модели**

In [ ]:
input1 = Input((xTrain.shape[1],))
input2 = Input((xBOW_text.shape[1],))

x1 = input1                               # Ветка 1
x1 = Dense(128, activation="relu")(x1)
x1 = Dense(1000, activation="tanh")(x1)
x1 = Dense(100, activation="relu")(x1)

x2 = input2                               # Ветка 2
x2 = Dense(20, activation="relu")(x2)
x2 = Dense(500, activation="relu")(x2)
x2 = Dropout(0.3)(x2)

x = concatenate([x1, x2])             # Объединение двух веток

x = Dense(15, activation='relu')(x)       # Промежуточный слой
x = Dropout(0.5)(x)
x = Dense(1, activation='linear')(x)      # Финальный регрессирующий нейрон

# В Model передаются входы и выход
model_final = Model((input1, input2), x)

model_final.compile(optimizer=Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])

In [ ]:
callback = ReduceLROnPlateau(monitor = 'val_loss',
                             factor = 0.2,
                             patience = 4,
                             min_lr = 0.000001)

In [ ]:
history = model_final.fit([xTrain, xBOW_text], yTrain,
                          batch_size = 128,
                          epochs =50,
                          validation_split = 0.2,
                          callbacks = [callback],
                          verbose = 1)

Epoch 1/50
328/328 [==============================] - 11s 11ms/step - loss: 7367.2124 - mae: 70.7209 - val_loss: 708.6855 - val_mae: 20.4510 - lr: 1.0000e-04
Epoch 2/50
328/328 [==============================] - 2s 6ms/step - loss: 2593.3481 - mae: 40.5105 - val_loss: 640.8026 - val_mae: 19.1961 - lr: 1.0000e-04
Epoch 3/50
328/328 [==============================] - 2s 7ms/step - loss: 2257.2976 - mae: 37.7958 - val_loss: 679.1077 - val_mae: 19.5562 - lr: 1.0000e-04
Epoch 4/50
328/328 [==============================] - 2s 6ms/step - loss: 2068.1804 - mae: 36.0059 - val_loss: 739.6730 - val_mae: 20.4706 - lr: 1.0000e-04
Epoch 5/50
328/328 [==============================] - 2s 6ms/step - loss: 2063.1758 - mae: 35.8621 - val_loss: 864.4452 - val_mae: 22.5162 - lr: 1.0000e-04
Epoch 6/50
328/328 [==============================] - 3s 9ms/step - loss: 2029.7068 - mae: 35.4789 - val_loss: 825.8398 - val_mae: 21.7383 - lr: 1.0000e-04
Epoch 7/50
328/328 [==============================] - 2s 6ms/s

In [ ]:
check_MAE_predictl_DubbleInput(model_final, xTrain, xBOW_text, yTrain)

1640/1640 [==============================] - 3s 2ms/step
Среднаяя абслолютная ошибка 14.497 очков это 9.561% от общей выборки в 52450 игры


Для решения задачи требуются знания по теме `Callbacks`, пожалуйста, изучите раздел:

---
Дополнителная информация: ([База знаний УИИ – **«Callbacks»**](https://colab.research.google.com/drive/1COE1R3kFhRlxPm9X6hQMvjKoiFFt83xW?usp=sharing))

---